In [4]:
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv
import logging

In [5]:
# Configure the logger
logging.basicConfig(
    level=logging.INFO,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s - Line: %(lineno)d',  # Define the output format
)

In [6]:
load_dotenv()

logger = logging.getLogger()

In [7]:
#Set Current Date and Date Format
date_format = '%d-%m-%Y'
changed_format ='%d/%m/%Y'
current_date = datetime.now().strftime(date_format)

In [8]:
#Google drive IDS for Files
raw_item_mst_csv_id = os.getenv('RAW_ITEM_MST_CSV_ID')
raw_cust_mst_csv_id = os.getenv('RAW_CUST_MST_CSV_ID')
state_code_mst_excel_id = os.getenv('STATE_CODE_MST_EXCEL_ID')
so_template_id = os.getenv('SO_TEMPLATE_ID')

In [32]:
#URL Creation
state_code_mst_excel = f'https://docs.google.com/spreadsheets/d/{state_code_mst_excel_id}/export?format=xlsx'

raw_cust_mst_csv = f'https://drive.google.com/uc?id={raw_cust_mst_csv_id}'

raw_item_mst_csv = f'https://drive.google.com/uc?id={raw_item_mst_csv_id}'

so_template = f'https://docs.google.com/spreadsheets/d/{so_template_id}/export?format=xlsx'

In [33]:
#Getting Input file
indent_data = input('Enter Indent file full Path (e.g., C:\\Users\\Downloads\\file.csv): ')

if not os.path.exists(indent_data):
    print('Please Enter correct path for the file!!!!')

In [34]:
#Reading SO Excel Template
#---------------------Reading SO Template------------------
logger.info('Reading SO Template')
raw_so_template = pd.read_excel(so_template, sheet_name='Input_Data')
#raw_so_template.head()

2024-10-20 22:36:54,035 - INFO - Reading SO Template


In [35]:
#Converting Cloumns to List
header = raw_so_template.columns.tolist()
#print(header)

In [36]:
#Creating a List to Exclude few Headers 
exclude_header_arr = ['External ID','CUSTOM FORM','ORDER TYPE','APPROVAL STATUS','SUBSIDIARY','LINE OF BUSINESS','BUSINESS VERTICALS','REMARKS','COMMIT','CLOSED','INDIA TAX NATURE','CURRENCY','DEPARTMENT','LINE OF BUSINESS ITEM','BUSINESS VERTICALS ITEM','DEPARTMENT ITEM']

#Creating a dict and setting fixed Values for Excluded Header Array
fvalues_exc_head = {
    'CUSTOM FORM' : 'BEPL Sales Order - Invoice', 'ORDER TYPE' : 'FOFO Indent', 'SUBSIDIARY' : 'Bloombay Enterprises Private Limited', 'LINE OF BUSINESS' : 'QSR', 'BUSINESS VERTICALS' : 'FOFO','DEPARTMENT' : 'Supply Chain',
    'REMARKS' : 'Indent to Sales Order Creation - Mannual', 'COMMIT' : 'Do Not Commit','CLOSED': 'NO', 'INDIA TAX NATURE' : 'Goods', 'CURRENCY' : 'INR',
    'Sales Order - Items : Line of Business': 'QSR',
    'Sales Order - Items : Business Verticals': 'FOFO',
    'Sales Order - Items : Department': 'Supply Chain'
}
#print(fvalues_exc_head)

In [37]:
# Filtering Excluded Header from Header
filtered_header = [h for h in header if h not in exclude_header_arr]
#print(filtered_header)

['ORDER ', 'CUSTOMER', 'Customer_Internal ID', 'DATE', 'PO #', 'INDENT NUMBER', 'SALES REP', 'SALES EFFECTIVE DATE', 'LOCATION', 'PAIRED INTERCOMPANY TRANSACTION', 'INTERCOMPANY STATUS E-COMMERCE OPERATOR', 'E-COMMERCE GSTIN', 'EXPORT TYPE', 'PLACE OF SUPPLY', 'LUT OR BOND NUMBER', 'MODE OF PAYMENT', 'ITEM', 'REQUESTED QUANTITY', 'QUANTITY', 'UNITS', 'DESCRIPTION', 'UNIT PRICE', 'Amount', 'LOCATION ITEM', 'INDIA TAX HSN OR SAC CODE']


In [38]:
#Creating Indnet and Sales Order Header Mapping
so_ind_map = {
    'Supplier ' : 'LOCATION', 'Receiver ' : 'CUSTOMER', 'Date' : 'DATE', 'Item Code ' : 'ITEM CODE', 'Item Name ' : 'ITEM NAME', 'Requested Qty ' : 'REQUESTED QUANTITY', 'Requested Qty ' : 'QUANTITY', 'Unit ' : 'UNITS', 'UnitPrice' : 'UNIT PRICE', 'SubTotal ' : 'Amount', 'Supplier ' : 'LOCATION ITEM'
}

#Importing Indent CSV
#----------------------Reading Indent CSV----------------
logger.info('Reading Indent File!')
indent_rawdata = pd.read_csv('indentReport.csv', usecols=lambda column: column in so_ind_map.keys())

#indent_rawdata.head()

2024-10-20 22:36:56,893 - INFO - Reading Indent File!


In [39]:
# Coping Filtered CSV for Concate Item Name and Item Code and dorpping Item Name, Item and Supplier Columns
indent_rawdata_item = indent_rawdata.copy()
indent_rawdata_item['ITEM'] = indent_rawdata_item['Item Code '] + ' ' + indent_rawdata_item['Item Name ']
#indent_rawdata_item.head()

#Droping Cloumns
indent_filtered_data = indent_rawdata_item.copy()

#Replacing Warehouse Names
for index, row in indent_filtered_data.iterrows():
    if row['Supplier '] == 'Central Warehouse - Mumbai - Bloombay Enterprise Private Limited':
        indent_filtered_data.at[index, 'LOCATION'] = 'BEPL Mumbai Warehouse'
    if row['Supplier '] == 'BWC-NCR-Delhi-WRH':
        indent_filtered_data.at[index, 'LOCATION'] = 'BEPL Delhi Warehouse'
    if row['Supplier '] == 'BWC-KA-Bengaluru-WRH':
        indent_filtered_data.at[index, 'LOCATION'] = 'BEPL Bengaluru Warehouse'

drop_cloumns = ['Item Code ', 'Item Name ','Supplier ']

#Drop Cloumns
indent_filtered_data.drop(columns= drop_cloumns ,inplace=True)
print('Indent Data Conversion Done!!!')

#indent_filtered_data.head()


Indent Data Conversion Done!!!


In [40]:
#Creating Columns names to be Updated List
updated_cloumn_name = ['CUSTOMER','DATE','UNITS','UNIT PRICE','REQUESTED QUANTITY','AMOUNT','ITEM','LOCATION']

updated_filtered_data = indent_filtered_data.copy()

#Updating Cloumns Names
updated_filtered_data.columns = updated_cloumn_name

#updated_filtered_data.head()

In [41]:
#Creating Dict of Indent Data to match data format for Fixed Values
dict_indent = []

for index, row in updated_filtered_data.iterrows():
    dict_indent.append(row.to_dict())

In [42]:
#Combining Indent and Fixed Values Dict
combined_dict = dict_indent.copy()

for d in combined_dict:
    d.update(fvalues_exc_head)


#print(combined_dict)


In [43]:
#Creating External ID

formatted_date = current_date.replace('-','')

for item in combined_dict:
    franchisee_code	= item['CUSTOMER'].split('-')[0].strip().replace('/', '')

    external_id = f'{franchisee_code}{formatted_date}'
    
    item['External ID'] = external_id



In [69]:
#Importing Item Master Data
#-----------------------------Reading Item Master CSV (NetSuite)----------
logger.info('Reading Item Master Data')
item_master_raw = pd.read_csv(raw_item_mst_csv)

In [34]:
# Concate Item Code and Name
item_master_concate = item_master_raw.copy()
item_master_concate['ITEM'] = item_master_concate['Name'] + ' ' + item_master_concate['Display Name']

#Cloumns Drop List
item_cols_drop = ['Name', 'Display Name']

final_item_mst = item_master_concate.copy()
final_item_mst.drop(columns=item_cols_drop, inplace=True)
final_item_mst.rename(columns={'Internal ID' : 'Item : Internal ID'}, inplace=True)

#final_item_mst.head()

In [35]:
#Importing Customer Master
#-------------------------------------Customer Master CSV Import-------
logger.info('Reading Customer Master Data!')
customer_master_raw = pd.read_csv(raw_cust_mst_csv)

#customer_master_raw.drop_duplicates(['Customer Name'], inplace=True)

#print(customer_master_raw.count())
#customer_master_raw.head()

In [36]:
#Import State Code Sheet 
state_codes =pd.read_excel(state_code_mst_excel)

#state_codes.head()

In [499]:
#Concate Customer ID and Name 
cust_mst_concate = customer_master_raw.copy()
cust_mst_concate['CUSTOMER'] = cust_mst_concate['Customer Name'] \
    .str.split('(') \
    .str[1] \
    .str.strip() \
    .str.replace(')', '' , regex=False)

cust_mst_join = pd.merge(cust_mst_concate, state_codes, on='GST_State_Code', how='inner')

final_cust_mst = cust_mst_join.copy()
cust_clons_drop = ['ID','GST_State_Code','Name']
final_cust_mst = final_cust_mst.drop(columns=cust_clons_drop)
final_cust_mst.rename(columns={'Internal ID' : 'Customer : Internal ID'}, inplace=True)

#print(cust_mst_filtered.count())
#final_cust_mst.head()


In [ ]:
print('Creating Sales Order Template')
#Creating Dataframe from Combined Dict 
final_dict = pd.DataFrame(combined_dict)

#Replacing Date format in Date Cloumn
final_dict['DATE'] = pd.to_datetime(final_dict['DATE'], format=date_format).dt.strftime(changed_format)

final_dict['Franchisee Code'] = final_dict['CUSTOMER'].str.split('-').str[0]

#final_dict.head()


In [501]:
join_item = pd.merge(final_dict, final_item_mst, on='ITEM', how='inner')
#join_item.head()

In [502]:
join_customer = pd.merge(join_item,final_cust_mst, on='CUSTOMER', how='inner')
join_customer.drop_duplicates('ITEM', inplace=True)

#print(join_customer.count())
#join_customer.head()


In [ ]:
export_data = join_customer.copy()
export_drop_cols =['Franchisee Code_x','Franchisee Code_y','CUSTOMER']
export_data.drop(columns=export_drop_cols, inplace=True)
export_data['Sales Order - Items : Location'] = export_data['LOCATION']
export_data['QUANTITY'] = export_data['REQUESTED QUANTITY']

#export_data.head()

In [ ]:
#Create file name
location_file = final_dict['LOCATION'].iloc[0]
file_name = f'SO_Indent_{location_file}_Dt.{current_date}.csv'

#Set Output path
output_dir = r'D:\Projects\Auto Output'

if os.path.exists(output_dir):
    output_path = os.path.join(output_dir, file_name)
else:
    user_download_path = os.path.join(os.path.expanduser("~"), 'Downloads')
    output_path = os.path.join(user_download_path,file_name)

In [ ]:
#Export Data to file
logger.info('Creating Sales Order Data')
export_data.to_csv(output_path,index=False)

logger.info(f'File Sucessfully Saved to {output_path}')
logger.info('Processing Completed!!!!')